In [ ]:
import numpy as np
import sys
sys.path.append("../")
from pathlib import Path

# Package for importing image representation
from PIL import Image, ImageOps

from src.compress_sensing_library import reconstruct, generate_classical_variables, error_calculation

import pandas as pd
import itertools
import dask
from dask.distributed import Client, progress
import time
import os.path

In [ ]:
def search(img_arr, repetition = 20, observation = 'classical' alpha_list = None, num_cell_list = None, method = 'dct'):
    cn, cm = img_arr.shape
    params = []
    results = []

    #Ternary Operater. Set up alpha and classical_samp_list to be default size for
    #Hyperparameter sweep
    alpha_list = np.logspace(-3, 3, 7) if None else alpha_list
    num_cell_list = [50, 100, 200, 500] if None else num_cell_list

    # List to make combination of
    search_list = {'repetition': repetition,
                   'alpha_list': alpha_list,
                   'classical_samp_list': num_cell_list
                  }



    # Get repetition * alpha combination amount of result
    for row in itertools.product(*search_list.values()):
        product = dict(zip(search_list.keys(), row))
        rep, alpha, sample_size = product.values()
        
        if 
        C3D, y = generate_classical_variables(img_arr, sample_size)
        theta, reconst, s = reconstruct(C3D, y, alpha)
        
        error = error_calculation(img_arr, reconst)
        
        params.append({'classical_samp' : classical_samp,
                       'alpha' : alp,
                       'method' : method,
                       'repetition' : rep,
                       'error' : error
                      })
        results.append({'s' : s,
                        'theta' : theta,
                        'reform' : reconst,
                       })
        
    return params, results, 


    
    

In [ ]:
def main():
    '''
    arg[1] = img_name ex)tree_part1.jpg
    arg[2] = observation_type ex)classical, V1, gaussian
    arg[3] = num_cell list ex) [50, 100, 200, 500]
    arg[4] = cell_size list ex) [1, 2, 4, 8, 10]
    arg[5] = sparse_freq ex) [1, 2, 4, 8, 10]
    '''
    file = sys.argv[1]
    image_path ='../image/{img}'.format(img = file)
    image_nm = image_path.split('/')[2].split('.')[0]
    img = Image.open(image_path)
    img = ImageOps.grayscale(img)
    img_arr = np.asarray(img)
    save_path = os.path.join("../result/{img_nm}/V1".format(img_nm = image_nm))
    params, results = search(img_arr = img_arr)
    print('Process Completed')
    
    save_path = os.path.join('./result/{img}/Classical/'.format(img = image_nm))

    classical_param_df = pd.DataFrame(params)
    classical_param_df.to_csv(os.path.join(save_path, "Classical_Param_" + "_".join(str.split(time.ctime().replace(":", "_"))) + ".csv"))
    print('Save Completed. Terminate program')

if __name__ == "__main__":
    main()
    

In [1]:
import numpy as np
import numpy.linalg as la
import sys
sys.path.append("../")
import matplotlib.pyplot as plt

# Package for importing image representation
from PIL import Image, ImageOps

from src.compress_sensing_library import *
import pandas as pd
import itertools
import dask
from dask.distributed import Client, progress
import time
import os.path



In [11]:
def run_sim(method, observation, rep, alpha, num_cell, cell_sz, sparse_freq, img_arr):
    num_cell = int(num_cell)
    img_arr = np.array([img_arr]).squeeze()
    dim = img_arr.shape
    n, m = dim

    
    if (observation.lower() == 'classical') :
        #TODO
        W, y = generate_classical_variables(img_arr, num_cell)
    elif (observation.lower() == 'gaussian') :
        #TODO
        W, y = generate_gaussian_variables(img_arr, num_cell)
    elif (observation.upper() == 'V1') :
        #TODO
        W, y = generate_V1_variables(img_arr, num_cell, cell_size, sparse_freq)
    
    
    
    # Call function and calculate error
    theta, reconst, s = reconstruct(W, y, alpha, method = method)
    error = error_calculation(img_arr, reconst)
    
    return error, theta, reconst, s


def main() :
    '''
    arg[1] = img_name ex)tree_part1.jpg
    arg[2] = method ex)dct, dwt
    arg[3] = observation_type ex)classical, V1, gaussian
    arg[4] = alpha_list ex) [0.001, 0.01, 0.1]
    arg[5] = num_cell list ex) [50, 100, 200, 500]
    arg[6] = cell_size list ex) [1, 2, 4, 8, 10]
    arg[7] = sparse_freq ex) [1, 2, 4, 8, 10]
    '''
    # Set up hyperparameters that would affect results
    param_dict = {
        1 : None, 
        2 : 'dct',
        3 : 'classical', 
        4 : np.logspace(-3, 3, 7),
        5 : [50, 100, 200, 500], 
        6 : [2, 5, 7],
        7 : [1, 2, 5]
    }
    if (len(sys.argv) < 1):
        print("File Name at least required");
    else :
        if (sys.argv[1] == "help" | sys.argv[1] == "-h"):
            print("Usage \n \
                   arg[1] = img_name ex)tree_part1.jpg \n \
                   \t arg[2] = method ex)dct, dwt\n \
                   \t arg[3] = observation_type ex)classical, V1, gaussian\n \
                   \t arg[4] = alpha_list ex) [0.001, 0.01, 0.1]\n \
                   \t arg[5] = num_cell list ex) [50, 100, 200, 500]\n \
                   \t arg[6] = cell_size list ex) [1, 2, 4, 8, 10]\n \
                   \t arg[7] = sparse_freq ex) [1, 2, 4, 8, 10]")
                  
            sys.exit(0)
        else :
            param_dict[1] = sys.argv[1]
    if (len(sys.argv) >= 2):
        for i in range(2, len(sys.argv)):
            if (sys.argv[i] != '-n')
            param_dict[i] = sys.argv[i]
    
    file, method, observation, alpha, num_cell, cell_sz, sparse_freq = param_dict.values()
    
    #Make sure the data type input is in correct format
    assert type(file) == str
    assert type(method) == str
    assert type(observation) == str
    assert type(alpha) == list
    assert type(num_cell) == list
    assert type(cell_sz) == list
    assert type(sparse_freq) == list
    
    
    image_path ='../image/{img}'.format(img = file)
    delay_list = []
    params = []
#     alpha = np.logspace(-3, 3, 7)
    rep = np.arange(20)
    

    image_nm = image_path.split('/')[2].split('.')[0]
    img = Image.open(image_path)
    img = ImageOps.grayscale(img)
    img_arr = np.asarray(img)
    #plt.imshow(img_arr)
    #plt.show()
    save_path = os.path.join("../result/{img_nm}/V1".format(img_nm = image_nm))

    # TODO: change search list and trailing to have observation, 
    #       and update observation to have function according to the its value
    
    search_list = [rep, alpha, num_cell, cell_sz, sparse_freq]

    # All combinations of hyperparameter to try 
    search = list(itertools.product(*search_list))             
    search_df = pd.DataFrame(search, columns= [ 'rep', 'alp', 'num_cell', 'cell_sz', 'sparse_freq'])
    print(search_df.head())

    # Call dask
    client = Client()

    # counter = 0; # Keep track of number of iteration. Debugging method
    for p in search_df.values:
        delay = dask.delayed(run_sim)(method, observation, *p, img_arr)
        delay_list.append(delay)

    print('running dask completed')

    futures = dask.persist(*delay_list)
    print('futures completed')
    progress(futures)
    print('progressing futures')

    # Compute the result
    results = dask.compute(*futures)
    print('result computed')
    results_df = pd.DataFrame(results, columns=['error', 'theta', 'reform', 's'])
    
    param_path = data_save_path(image_nm, method, observation, "param_" + "_".join(str.split(time.ctime().replace(":", "_"))))
    result_path = data_save_path(image_nm, method, observation, "result_" + "_".join(str.split(time.ctime().replace(":", "_"))))
    # Add error onto parameter
    params_result_df = search_df.join(results_df['error'])
    
    # save parameter_error data with error_results data
    params_result_df.to_csv(param_path)
    results_df.to_csv(result_path)
    print("Execution Complete")



In [3]:
if __name__ == "__main__":
    main()

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [5]:
data_save_path('tree_part1', 'dct', 'classical', "param_" + "_".join(str.split(time.ctime().replace(":", "_"))))

'../result/dct/tree_part1/classical/param_Mon_Mar_20_17_42_40_2023.csv'

In [9]:
dict = {
    1: [0, 1, 2],
    2: 1,
    3: 2,
}

In [ ]:
dict

In [7]:
dict[1] = 0

In [10]:
dict[1]

[0, 1, 2]

In [ ]:
assert type('type(dict[1])') == str

In [ ]:
type("adf")

In [ ]:
b

In [ ]:
arg = 4
arg1 = 5
arg2 = 6
a = 1
b = 2
c = 3
d = 1

In [ ]:
h = arg if a == b else arg1 if b == c else arg2 if a == d else 0
print(h)

In [ ]:
def hello(a, b):
    return a+b

In [ ]:
e = hello

In [1]:
dict = {}

In [3]:
dict[1] = 1

In [8]:
lst = [1, 2, 3, 4, 5]
lst[0]

1

In [11]:
#lst = None

if (True):
    st = 1
st

1

In [13]:
lst = 10
len(lst)

TypeError: object of type 'int' has no len()